<p>
<font size='5' face='Georgia, Arial'>IIC2115 - Programación como herramienta para la ingeniería</font><br>
<font size='1'>Basado en material de Karim Pichara y Christian Pieringer. Todos los derechos reservados.</font>
</p>

<h1>Herencia</h1>

El concepto de herencia en programación orientada a objetos nos permite aprovechar código de las clases de las cuales se hereda. La herencia nos permite representar la relación del tipo "el objeto B es un objeto A, pero con ciertas diferencias".

Una clase hija (o subclase) corresponde a una <b>especialización</b> de su clase padre. Cuando un objeto pertenece a una clase en particular, si esta clase es a su vez una subclase de otra clase más general, la herencia nos permite "heredar" los datos y comportamiento de la clase "madre" (superclase), de tal manera de no tener que volver a definir esos datos y comportamiento en la subclase. Por ejemplo: La clase "furgón escolar" es una subclase de la clase "vehículo", por lo tanto sabemos que la clase "furgón escolar" va a heredar los datos y comportamiento de "vehículo" (ruedas, motor, etc.) y no es necesario volver a definirlos en la subclase "furgón escolar". Lo interesante es que la subclase "furgón escolar" tiene ciertos datos y métodos que la hacen más especializada que la clase "vehículo", <i>i.e.</i>, lista de niños inscritos en el furgón.

También la herencia nos permite sobrescribir los métodos que necesitemos modificar (_method overriding_). En Python, simplemente definimos nuevamente el método y con eso se entiende que la versión implementada en la subclase es la que cuenta. Una de las cosas que podemos hacer con herencia es extender los elementos y estructuras que provee el lenguaje (_built-ins_), por ejemplo, si queremos extender la clase "lista", podemos definir una subclase que heredará los métodos de la clase "lista" y a su vez tendrá datos y métodos propios:


In [ ]:
# Aquí estamos extendiendo y especializando la clase lista estándar. Tiene todos los métodos de la lista más los definidos por
# nosotros. Recordar que para nombrar las clases se utiliza notación CamelCase.
class ContactList(list):
    
    # buscar es un método específico de esta sub-clase
    def buscar(self, nombre):
        matches = []
        
        for contacto in self:
            if nombre in contacto.nombre:
                matches.append(contacto)
                
        return matches

    
class Contacto:
    
    # Contacto se compone de una lista de contactos del tipo ContactList
    
    # notar que contactos_list es un atributo estático, o sea, es compartido por todos los objetos de la clase Contacto
    # contactos_list = [] #así sería para usar una lista común y corriente    
    contactos_list = ContactList()

    def __init__(self, nombre, email):
        self.nombre = nombre
        self.email = email
        Contacto.contactos_list.append(self) # el método append() es heredado de la clase List()


# Familiar es una clase especializada de contacto que permite incluir el tipo de relación
class Familiar(Contacto):

    def __init__(self, nombre, email, relacion): # Overriding sobre el método __init__()
        super().__init__(nombre, email) # Obtiene la instancia del padre y llama a su funcion __init__
        self.relacion = relacion

In [ ]:
p1 = Familiar(nombre = "Juan Gómez", email = "jg@hotmail.com", relacion = "padre")
p2 = Contacto(nombre = "Jorge González", email = "jg@gmail.com")
p3 = Familiar(nombre = "Pablo Gómez", email = "pab_g@gmail.com", relacion = 'primo')
p4 = Contacto(nombre = "Jorge Contreras", email = "jc@gmail.com")

L = [c.nombre for c in p1.contactos_list.buscar("Jorge")]

print('[', end='')
print(*L, sep=', ', end='')
print(']')

<h1>Polimorfismo</h1>

El polimorfismo se refiere a "la propiedad de enviar mensajes sintácticamente iguales a objetos de tipos distintos" ([Wikipedia](https://es.wikipedia.org/wiki/Polimorfismo_(inform%C3%A1tica))). Básicamente es utilizar objetos de tipos distintos (instancias de distintas clases) con la misma _interfaz_. _Overriding_ (sobreescritura) es la forma más típica de aplicar polimorfismo y ocurre cuando se implementa un método en una subclase que "invalida" la implementación del mismo método en la super clase.
     
#### Ejemplo

La clase `Variable` representa un conjunto de datos cualquiera, mientras que la subclase `Ingresos` contiene un método para calcular el valor "representante" (algo así como el promedio, mediana, moda, etc.). Ocurre lo mismo con las subclases `Comuna` y `Puesto`: Si los datos corresponden a ingresos, el representante es el promedio. Si los datos corresponden a la comuna, el representante es la comuna que más se repite. Finalmente, si los datos corresponden al puesto de trabajo, entonces el representante es el que tiene el puesto más alto según la jerarquía especificada en el diccionario "categorías".

In [ ]:
import numpy as np  #veremos numpy en detalle más adelante, por el momento sólo lo utilizaremos

class Variable:
    
    def __init__(self, data):
        self.data = np.array(data)

    def representante(self):
        pass


class Ingresos(Variable):
    
    def representante(self):
        return np.mean(self.data)


class Comuna(Variable):
    
    def representante(self):
        ind = np.argmax([np.sum(self.data == c) for c in self.data])  # el que mas se repite
        return self.data[ind]


class Puesto(Variable):
    
    categorias = {'Gerente': 1, 'SubGerente': 2, 'Analista': 3, 
                  'Alumno en Practica': 4} # class (or static) variable

    def representante(self):
        return self.data[np.argmin([Puesto.categorias[c] for c in self.data])]#la categoria mas alta acorde con el diccionario

In [ ]:
lista_pesos = Ingresos([50, 80, 90, 150, 45, 65, 78, 89, 59, 77, 90])
lista_comunas = Comuna(['Providencia', 'Macul' , 'LaReina' ,'Santiago', 'Providencia', 'PuenteAlto',
                        'Macul', 'Santiago', 'Santiago' ])
lista_puestos = Puesto(['SubGerente', 'Analista','SubGerente','Analista','Alumno en Practica',
                        'Alumno en Practica'])


print(lista_pesos.representante())
print(lista_comunas.representante())
print(lista_puestos.representante())

En este último ejemplo podemos apreciar claramente como es posible reutilizar de manera eficiente una misma interfaz, para entregar resultados distintos (que además son generados a partir de dominios distintos)

<h2> Overriding de operadores en Python </h2>

Existen muchos operadores en Python que funcionan para varias de las clases "built-in". Por ejemplo, el operador "+" puede sumar dos números, concatenar dos strings, mezclar dos listas, etc. dependiendo de la clase con la que estemos trabajando:

In [ ]:
a = [1,2,3,4]
b = [5,6,7,8]
print(a+b)
c = "Hola"
d = " Mundo"
print(c+d)

Nosotros también podemos personalizar el método `__add__` para que funcione en algún tipo de clase específica que necesitemos. Por ejemplo, supongamos una clase que representa un carro de compra:

In [ ]:
class Carro:
    '''
    Un carro de compras lo representaremos como un diccionario 
    donde el key es el nombre del producto y el value es la cantidad
    Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():  # aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})  # aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        return "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())

In [ ]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3.lista_productos)

 El método `__repr__` nos permite generar un string que será usado a la hora de llamar a print de alguna instancia de Carro:

In [ ]:
print(carro_3)

También podríamos haber implementado el método `__str__` que cumple la misma función que el método `__repr__`. La principal diferencia es que `__repr__` debería contener todos los detalles necesarios para identificar bien al objeto, como para ser usado por alguien que implementará algo en el futuro y debe entender bien nuestro código. El método `__str__` está orientado a generar una impresión "human-readable", algo que se vea bien y se interprete bien en el contexto en particular, pero no necesariamente debe contener todos los detalles técnicos del objeto. En casos en que `__str__` está implementado, print usará el string generado por `__str__` para imprimir, pero cuando no esté `__str__` implementado, print usará el método `__repr__`.

In [ ]:
class Carro:
    ''' Un carro de compras lo representaremos como un diccionario 
        donde el key es el nombre del producto y el value es la cantidad
        Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():#aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})#aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        s = self.__doc__#esto retorna el string del comienzo de la clase, la documentación que la describe
        return s + "\n" + "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())
    
    def __str__(self):
        return "\n".join("Producto: {} - Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())    

In [ ]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3)

Si comentamos el método `__str__`, print va a imprimir el string que retorna la función `__repr__`

In [ ]:
class Carro:
    ''' Un carro de compras lo representaremos como un diccionario 
        donde el key es el nombre del producto y el value es la cantidad
        Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():#aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})#aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        s = self.__doc__#esto retorna el string del comienzo de la clase, la documentación que la describe
        return s + "\n" + "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())
    
#    def __str__(self):
#        return "\n".join("Producto: {} - Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())    

In [ ]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3)

De la misma forma podemos personalizar la mayoría de los operadores, por ejemplo, para personalizar el operador "menor que" (less than):

In [ ]:
class Punto: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y
    
    def __lt__(self, otro_punto): 
        self_mag = (self.x ** 2) + (self.y ** 2) 
        otro_punto_mag = (otro_punto.x ** 2) + (otro_punto.y ** 2) 
        return self_mag < otro_punto_mag

p1 = Punto(2,4)
p2 = Punto(8,3)
print(p1 < p2)

# Diagrama de Clases

Los diagramas de clases corresponden a una herramienta que nos permite visualizar fácilmente las clases que componen un sistema, así como también sus propiedades, métodos, relaciones e interacciones que existen entre ellas. 

## Elementos de un diagrama de Clases

Un diagrama de clases se compone de clases y relaciones:


### Clases

Cada clase de un sistema debe representarse de manera independiente, encapsulando toda su información. Gráficamente, una clase se representa con un rectángulo dividido en tres niveles. El primer nivel contiene el nombre de la clase; el segundo contiene los atributos o variables propias de la clase; y, finalmente, el tercero contiene los métodos propios de la clase. 

![](figs/UML_class.png)


Como ejemplo, consideremos el caso de un catálogo de objetos estelares, que debe ser modelado utilizando clases. Un catálogo agrupa un conjunto de estrellas pertenecientes a una determinada galaxia. Cada estrella se representa como una serie de tiempo,
formada por un conjunto de observaciones, que corresponden a la magnitud del brillo de una estrella a lo largo del tiempo, con un error asociado a la medición: $\{ (m_1, t_1, e_1), (m_2, t_2, e_2), \ldots, (m_T, t_T, e_T) \}$. Cada serie de tiempo tiene además un número identificador. 

Usando diagramas de clases, podemos modelar este sistema como muestra en la siguiente figura.

![](figs/UML_catalog.png)
![](figs/UML_star.png)
![](figs/UML_observation.png)

Como se puede observar, para los atributos se debe especificar su nombre y tipo de variable. Por otro lado, para los métodos se debe especificar su nombre y el tipo de variable esperado para su valor de retorno.


Supongamos también que la clase SerieDeTiempo posee métodos para:

- agregar una observación
- retornar el promedio y desviación estándar de observaciones registradas.


Gráficamente, podemos representar este requerimiento como muestra la siguiente figura:

![](figs/UML_star_method.png)




### Relaciones

Los diagramas de clases explican cómo ocurre la interacción entre las clases dentro del sistema que modelamos. Las relaciones más comunes son: **composición**, **agregación** y **herencia**.


#### Composición:

En este tipo de relación, los objetos de la clase que creamos se contruye a partir de la inclusión de otros elementos. El tiempo de vida del objeto que componemos está condicionado por el tiempo de vida del objeto que lo incluye. En otras palabras, **la existencia de los objetos incluidos depende del objeto que los incluye.** La relación entre las clases se indica por una flecha que parte desde el objeto base y va hasta el objeto que componemos. La base de la flecha es un rombo **relleno**. Como ejemplos, consideremos el caso del objeto SerieDeTiempo, en que la serie se compone de un conjunto de observaciones. La composición se representa gráficamente como muestra la figura:

![](figs/UML_composition.png)

#### Agregación:

En este tipo de relación, la clase también es construida usando otros objetos, pero en este caso, el tiempo de vida del objeto que agregamos es independiente del tiempo de vida del objeto que lo incluye. La asociación entre los objetos se indica por una flecha que parte desde el objeto base y va hasta el objeto que agregamos. A diferencia de la composición, la base de la flecha es un rombo **sin rellenar**. Consideremos el caso del objeto *Catalogo*, el cual se compone de un conjunto de estrellas. En este caso, es posible apreciar que las estrellas pueden existir por si solas como un objeto independiente del catálogo. La composición se representa gráficamente como muestra la figura:

![](figs/UML_aggregation.png)

#### Herencia:

La relación de herencia se define gráficamente con una flecha de punta vacía que apunta hacía la super clase, como muestra la siguiente figura, donde se tiene la superclase SerieDeTiempo y su subclase SerieDeTiempoPeriodica, que presenta una especialización de la serie de tiempo que fue definida inicialmente: 

![](figs/UML_inheritance.png)


#### Modelo completo

Podemos entonces modelar completamente el problema descrito anteriormente usando diagramas de clases como muestra la siguiente figura:

![](figs/UML_diagram.png)